In [1]:
from keras.layers import Dense, Dropout, BatchNormalization, Input
from keras.models import Model
from keras.utils import multi_gpu_model, to_categorical
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import re as re
import seaborn as sns

Using TensorFlow backend.


In [63]:
train = pd.read_csv('train.csv')
print(train.isnull().sum()/len(train)*100)
train.head()
test = pd.read_csv('test.csv')
print(test.isnull().sum()/len(test)*100)
df = [train, test]

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64
PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Cabin          78.229665
Embarked        0.000000
dtype: float64


In [58]:
df = train
print(df.Embarked.mode())
Y_train = df.Survived.to_numpy().reshape((-1,1))
print(df.Cabin.unique())
print(Y_train[:10])

0    S
dtype: object
[nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34'
 'C104' 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79'
 'E25' 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68'
 'A10' 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23' 'B50' 'A26' 'D48' 'E58'
 'C126' 'B71' 'B51 B53 B55' 'D49' 'B5' 'B20' 'F G63' 'C62 C64' 'E24' 'C90'
 'C45' 'E8' 'B101' 'D45' 'C46' 'D30' 'E121' 'D11' 'E77' 'F38' 'B3' 'D6'
 'B82 B84' 'D17' 'A36' 'B102' 'B69' 'E49' 'C47' 'D28' 'E17' 'A24' 'C50'
 'B42' 'C148']
[[0]
 [1]
 [1]
 [1]
 [

In [59]:
def extractTitle(name):
    #input a name and output the title in the name
    title = re.search(' ([A-Za-z]+)\. ', name)
    if title:
        #if title found, return title
        return title.group(1)
    return "No title"

df['Title'] = df['Name'].apply(extractTitle)
test['Title'] = test['Name'].apply(extractTitle)

print(pd.crosstab(df['Title'], df['Sex']))
print(df.Title.unique())

Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1
['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']


In [60]:
df.Title = df.Title.replace(['Capt', 'Col', 'Countess', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Rev', 'Sir'], 'Rare')
df.Title = df.Title.replace(['Mlle', 'Ms'], 'Miss')
df.Title = df.Title.replace('Mme', 'Mrs')

In [61]:
#Fill missing age
def fillAge(df):
    fillAge_df = df
    
    avgAge = fillAge_df.Age.mean()
    stdAge = fillAge_df.Age.std()
    ageNullCount = fillAge_df.Age.isnull().sum()
    
    #generate random ages based on avg and std
    ageRandomList = np.random.randint(avgAge-stdAge, avgAge+stdAge, size=ageNullCount)
    fillAge_df['Age'][np.isnan(fillAge_df['Age'])] = ageRandomList
    fillAge_df['Age'] = fillAge_df['Age'].astype(int)
    
    return fillAge_df

df = fillAge(df)

/home/jason/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [62]:
#print rates of survival for all data columns
print(df[['Title', 'Survived']].groupby('Title', as_index=False).mean())
print('\n')
print(df[['Pclass', 'Survived']].groupby('Pclass', as_index=False).mean())
print('\n')
print(df[['Sex', 'Survived']].groupby('Sex', as_index=False).mean())
print('\n')
df['CategoricalFare'] = pd.qcut(df['Fare'], 4)
df['CategoricalAge'] = pd.qcut(df['Age'], 5)
print(df[['CategoricalFare', 'Survived']].groupby('CategoricalFare', as_index=False).mean())
print('\n')
print(df[['CategoricalAge', 'Survived']].groupby('CategoricalAge', as_index=False).mean())
print('\n')
df.Embarked = df.Embarked.fillna(df.Embarked.mode())
print(df[['Embarked', 'Survived']].groupby('Embarked', as_index=False).mean())
print('\n')
df['IsNotAlone'] = 1
df.loc[df['SibSp'] + df['Parch'] == 0, 'IsNotAlone'] = 0
print(df[['IsNotAlone', 'Survived']].groupby('IsNotAlone', as_index=False).mean())
df.Cabin = df.Cabin.fillna('U')
df['HasCabin'] = 1
df.loc[df['Cabin']=='U', 'HasCabin'] = 0
print(df[['HasCabin', 'Survived']].groupby('HasCabin', as_index=False).mean())

    Title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


      Sex  Survived
0  female  0.742038
1    male  0.188908


   CategoricalFare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


   CategoricalAge  Survived
0  (-0.001, 19.0]  0.460000
1    (19.0, 25.0]  0.311765
2    (25.0, 32.0]  0.377049
3    (32.0, 40.0]  0.375723
4    (40.0, 80.0]  0.381818


  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.336957


   IsNotAlone  Survived
0           0  0.303538
1           1  0.505650
   HasCabin  Survived
0         0  0.299854
1         1  0.666667


In [25]:
print(df.columns)
df = df.drop(['CategoricalFare', 'CategoricalAge'], axis=1)

def preprocessData(df):
    process_df = df
    
    #remove unnecessary columns
    process_df = process_df.drop(['PassengerId', 'Survived', 'Name', 'Age', 'Sibsp', 'Parch', 'Ticket', 'Cabin'], axis=1)
    
    #process string data into numerical data
    #higher survival rate categories should be given higher numerical numbers to be positively correlated with survival
    title_mapping = {'Mr':0, 'Rare':1, 'Master':2, 'Miss':3, 'Mrs':4}
    process_df['Title'] = process_df['Title'].map(title_mapping)
    
    sex_mapping = {'male':0, 'female':1}
    process_df['Sex'] = process_df['Sex'].map(sex_mapping)
    
    embarked_mapping = {'S':0, 'Q':1, 'C':2}
    process_df['Embarked'] = process_df['Embarked'].map(embarked_mapping)
    
    process_df.loc[process_df['Fare'] <= 7.91, 'Fare'] = 0
    process_df.loc[process_df['Fare'] > 7.91 & process_df['Fare'] <= 14.454, 'Fare'] = 1
    process_df.loc[process_df['Fare'] > 14.454 & process_df['Fare'] <= 31.0, 'Fare'] = 2
    process_df.loc[process_df['Fare'] > 31.0, 'Fare'] = 3
    
    process_df.loc[process_df['Age'] > 19.0 & process_df['Age'] <= 25.0, 'Age'] = 0
    process_df.loc[process_df['Age'] > 32.0 & process_df['Age'] <= 40.0, 'Age'] = 1
    process_df.loc[process_df['Age'] > 25.0 & process_df['Age'] <= 32.0, 'Age'] = 2
    process_df.loc[process_df['Age'] > 40.0 & process_df['Age'] <= 80.0, 'Age'] = 3
    process_df.loc[process_df['Age'] <= 19.0, 'Age'] = 4
    
    return process_df

df = preprocessData(df)
test = preprocessData(test)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title',
       'CategoricalFare', 'CategoricalAge', 'IsAlone'],
      dtype='object')

In [ ]:
df.corr()

In [ ]:
Y_train = np.array(df.Survived).reshape((-1, 1))
Y_train[:10]
df = df.drop(['Survived', 'Pclass'], axis=1)

In [ ]:
randforestregressor = RandomForestRegressor(max_depth=10, random_state=1, verbose=1)
df = pd.get_dummies(df)
randforestregressor.fit(df, train.Survived)

https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/
Code for RandomForestRegressor was taken from here as well as ideas for feature selection and dimensionality reduction techniques.

In [ ]:
#look at feature importance
features = df.columns
importances = randforestregressor.feature_importances_
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
def buildModel():
    # NEED TO EXPERIMENT WITH MODEL ARCHITECTURES
    # FIRST TEST IS NUMBER OF DENSE LAYERS FROM 0 - 5
    num_models = 6
    model = [0] * num_models
    
    for i in range(num_models):
        inputs = Input(shape=(5,))
        
        X = inputs
        
        for j in range(i):
            X = Dense(8, activation='relu', kernel_regularizer='l2')(X)
        
        outputs = Dense(1, activation='sigmoid', kernel_regularizer='l2')(X)
        
        model[i] = multi_gpu_model(Model(inputs=inputs, outputs=outputs), gpus=2) #initialise model with 2 GPUs
        model[i].compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = buildModel()
model[0].summary()

In [ ]:
X_train = df.to_numpy()

# REDUCE LEARNING RATE ON PLATEAU
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

In [ ]:
# EARLY STOPPING
early_stop = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=11, restore_best_weights=True, verbose=1)

history = [0] * 6
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size=0.1)

for i in range(6):
    history[i] = model[i].fit(x=X_train2, y=Y_train2, batch_size=128, epochs=100, callbacks=[reduce_lr], validation_data=[X_val2, Y_val2], verbose=0)
    print("NN {0:d}: No. dense layers = {1:d}, Train accuracy = {2:.5f}, Validation accuracy = {3:.5f}".format(i+1, i, max(history[i].history['acc']), max(history[i].history['val_acc'])))

Based on above results, 4 is the best number of dense layers. Next is experimenting with number of neurons in each layer.

In [ ]:
def buildModel2():
    # NEED TO EXPERIMENT WITH MODEL ARCHITECTURES
    # SECOND TEST IS NUMBER OF NEURONS IN EACH LAYER FROM 8-128
    num_models = 5
    model = [0] * num_models
    
    for i in range(num_models):
        inputs = Input(shape=(5,))
        
        X = inputs
        
        for j in range(4):
            X = Dense(8*(2**(i+j)), activation='relu', kernel_regularizer='l2')(X)
        
        outputs = Dense(1, activation='sigmoid', kernel_regularizer='l2')(X)
        
        model[i] = multi_gpu_model(Model(inputs=inputs, outputs=outputs), gpus=2) #initialise model with 2 GPUs
        model[i].compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = buildModel2()
model[0].summary()

In [ ]:
history = [0] * 5
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size=0.1)

for i in range(5):
    history[i] = model[i].fit(x=X_train2, y=Y_train2, batch_size=128, epochs=100, callbacks=[reduce_lr], validation_data=[X_val2, Y_val2], verbose=0)
    print("NN {0:d}: No. nodes in layer = {1:d}, Train accuracy = {2:.5f}, Validation accuracy = {3:.5f}".format(i+1, 8*(2**i), max(history[i].history['acc']), max(history[i].history['val_acc'])))

Overall 32 nodes in each layer is the most consistent.

In [ ]:
def buildModel3():
    # BUILD 15 MODELS OF ARCHITECTURE INPUT => D32 => D64 => D128 => D256 => OUTPUT
    num_models = 15
    model = [0] * num_models
    
    for i in range(num_models):
        inputs = Input(shape=(5,))
        
        X = inputs
        
        X = Dense(32, activation='relu', kernel_regularizer='l2')(X)
        X = Dense(64, activation='relu', kernel_regularizer='l2')(X)
        X = Dense(128, activation='relu', kernel_regularizer='l2')(X)
        X = Dense(256, activation='relu', kernel_regularizer='l2')(X)
        
        outputs = Dense(1, activation='sigmoid')(X)
        
        model[i] = multi_gpu_model(Model(inputs=inputs, outputs=outputs), gpus=2) #initialise model with 2 GPUs
        model[i].compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = buildModel3()
model[0].summary()

In [ ]:
history = [0] * 15

for i in range(15):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size=0.1)
    history[i] = model[i].fit(x=X_train2, y=Y_train2, batch_size=128, epochs=100, callbacks=[reduce_lr], validation_data=[X_val2, Y_val2], verbose=0)
    print("NN {0:d}: Train accuracy = {1:.5f}, Validation accuracy = {2:.5f}".format(i+1, max(history[i].history['acc']), max(history[i].history['val_acc'])))

In [ ]:
test = pd.read_csv('test.csv')
test.head()

In [ ]:
def dataPreprocess(df, meanAge):
    preprocessed_df = df
    preprocessed_df.Age = preprocessed_df.Age.fillna(meanAge)
    preprocessed_df = preprocessed_df.drop(['Cabin', 'Embarked', 'Ticket', 'Name', 'PassengerId', 'Pclass'], axis=1)
    preprocessed_df.Sex = preprocessed_df.Sex.map({'female': 1, 'male': 0})
    
    return preprocessed_df

In [ ]:
test_df = dataPreprocess(test, meanAge)
X_test = test_df.to_numpy()
print(X_test[:10])

In [ ]:
#predict
results = np.zeros((X_test.shape[0],1))
for i in range(15):
    results = results + model[i].predict(X_test)
results = pd.Series(np.rint(results/15).astype(int).reshape(X_test.shape[0]), name='Survived')
passengerid = test.PassengerId
submission = pd.concat([passengerid, results], axis=1)
submission.to_csv('ENSEMBLE_DENSE_L2.csv', index=False)